#### This notebook creates the data files for the assignment
#### I have found no way to make the original notebook generate the data files after running the first breakout section.

In [1]:
# ! pip install langchain==0.2.16
# ! pip install langchain-cohere==0.3.0
# ! pip install langchain-community==0.2.17
# ! pip install langchain-core==0.2.41
# ! pip install langchain-experimental==0.3.2
# ! pip install langchain-huggingface==0.1.0
# ! pip install langchain-openai==0.1.25
# ! pip install langchain-qdrant==0.1.3
# ! pip install langchain-text-splitters==0.2.4
# ! pip install langgraph==0.2.16
# ! pip install langgraph-checkpoint==1.0.6
# ! pip install langsmith==0.1.129
# ! pip install ragas==0.1.20

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

In [3]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 26, 13, 29, 40, 363941)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

In [17]:
from langchain_community.vectorstores  import Qdrant
from langchain_openai import OpenAIEmbeddings



embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

In [18]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

In [19]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [20]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

In [21]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [4]:
# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator_llm = ChatOpenAI(model="gpt-4o-mini")
# critic_llm = ChatOpenAI(model="gpt-4o")
# embeddings = OpenAIEmbeddings()

# generator = TestsetGenerator.from_langchain(
#     generator_llm,
#     critic_llm,
#     embeddings
# )

# distributions = {
#     simple: 0.5,
#     multi_context: 0.4,
#     reasoning: 0.1
# }

In [5]:
# testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=False)

embedding nodes:   0%|          | 0/200 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 1, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Exhilarating', 'Amazing action film', 'Fun factor', 'Highest of any movie this year']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Non-creative copying of past movies', 'Stereotypic bad guys', 'Master killer with hidden arsenal', 'Reeves character and tattoos', 'Record number of kills in a movie']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 1, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Worst movie of the franchise', 'Over the top', 'Magical jackets', 'Stop the bullets']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 2, 'structure': 2, 'relevance': 3, 'score

In [22]:
for data_row in testset.test_data:
    question = data_row.question
    contexts = data_row.contexts
    ground_truth = data_row.ground_truth
    evolution_type = data_row.evolution_type
    metadata = data_row.metadata
    
    # Process each element as needed
    print(f"Question: {question}")
    print(f"Contexts: {contexts}")
    print(f"Ground Truth: {ground_truth}")
    print(f"Evolution Type: {evolution_type}")
    print(f"Metadata: {metadata}")
    print("\n")  # For better readability

Question: What role does the Russian mafia play in John Wick's story?
Contexts: [": 20\nReview: After resolving his issues with the Russian mafia, John Wick (Keanu Reeves) returns home. But soon the mobster Santino D'Antonio (Riccardo Scamarcio) visits him to show Wick's marker and tells that he needs to help. John Wicks refuses since he is retired and Santino blows-up his house. John Wick meets the owner of the Continental hotel in New York City, Winston (Ian McShane), and he tells that Wick cannot violate the Mafia rules and shall honor the marker. Santino asks John Wick to kill his sister Gianna D'Antonio (Claudia Gerini) in Rome so that he can sit on the High Table of the criminal organizations. When John Wicks accomplishes his assignment, Santino puts a seven-million dollar contract on him attracting professional killers from everywhere. But Wick promises to kill Santino that is not protected by his marker anymore."]
Ground Truth: The Russian mafia plays a significant role in John

In [23]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does the Russian mafia play in John ...,[: 20\nReview: After resolving his issues with...,The Russian mafia plays a significant role in ...,simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
1,What is the nature of the review for John Wick...,[: 7\nReview: This review of John Wick: Chapte...,The review of John Wick: Chapter 2 is spoiler ...,simple,"[{'source': 'john_wick_2.csv', 'row': 7, 'Revi...",True
2,What is the role of The Marquis in the conflic...,"[: 1\nReview: The Table, the international crm...",The Marquis is empowered by The Table to deal ...,simple,"[{'source': 'john_wick_4.csv', 'row': 1, 'Revi...",True
3,What role does the pulsating score play in enh...,[: 13\nReview: Following on from two delirious...,The answer to given question is not present in...,simple,"[{'source': 'john_wick_3.csv', 'row': 13, 'Rev...",True
4,What distinguishes the fast paced action in th...,[: 5\nReview: The first John Wick film was spe...,The fast paced action in the first John Wick f...,simple,"[{'source': 'john_wick_3.csv', 'row': 5, 'Revi...",True
5,What are the characteristics that make 'John W...,[: 19\nReview: I really don't understand the l...,The characteristics that make 'John Wick' feel...,simple,"[{'source': 'john_wick_1.csv', 'row': 19, 'Rev...",True
6,What is the criticism of computer-generated im...,[: 10\nReview: Most American action flicks rel...,The criticism of computer-generated imagery in...,simple,"[{'source': 'john_wick_4.csv', 'row': 10, 'Rev...",True
7,What themes related to consequences are explor...,[: 24\nReview: John Wick: Chapter 3 - Parabell...,The themes related to consequences explored in...,simple,"[{'source': 'john_wick_3.csv', 'row': 24, 'Rev...",True
8,What role do magical jackets play in the movie?,[: 7\nReview: In my opinion this is by far the...,The answer to given question is not present in...,simple,"[{'source': 'john_wick_4.csv', 'row': 7, 'Revi...",True
9,What role do the action sequences play in maki...,"[: 9\nReview: At first glance, John Wick sound...",The action sequences in John Wick are pivotal ...,simple,"[{'source': 'john_wick_1.csv', 'row': 9, 'Revi...",True


In [24]:
from langsmith import Client

client = Client()

## commented the following code because has already run once and will mess up Langsmith

# dataset_name = "John Wicks Dataset"

# dataset = client.create_dataset(
#     dataset_name=dataset_name,
#     description="Questions about John Wicks movie reviews"
# )

In [25]:
for test in testset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": test[1]["question"]
      },
      outputs={
          "answer": test[1]["ground_truth"]
      },
      metadata={
          "context": test[0]
      },
      dataset_id=dataset.id
  )

In [26]:
testset.test_data[0]

DataRow(question="What role does the Russian mafia play in John Wick's story?", contexts=[": 20\nReview: After resolving his issues with the Russian mafia, John Wick (Keanu Reeves) returns home. But soon the mobster Santino D'Antonio (Riccardo Scamarcio) visits him to show Wick's marker and tells that he needs to help. John Wicks refuses since he is retired and Santino blows-up his house. John Wick meets the owner of the Continental hotel in New York City, Winston (Ian McShane), and he tells that Wick cannot violate the Mafia rules and shall honor the marker. Santino asks John Wick to kill his sister Gianna D'Antonio (Claudia Gerini) in Rome so that he can sit on the High Table of the criminal organizations. When John Wicks accomplishes his assignment, Santino puts a seven-million dollar contract on him attracting professional killers from everywhere. But Wick promises to kill Santino that is not protected by his marker anymore."], ground_truth="The Russian mafia plays a significant ro

In [27]:
testset_df = testset.to_pandas()
testset_df.to_csv("testset.csv")

In [28]:
import pandas as pd

test_df = pd.read_csv("testset.csv")

In [29]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [30]:
answers = []
contexts = []

for question in test_questions:
  response = naive_retrieval_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [31]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [32]:
response_dataset[0]

{'question': "What role does the Russian mafia play in John Wick's story?",
 'answer': "The Russian mafia plays a significant role in John Wick's story, particularly in the first movie where the main conflict is driven by a young Russian-American punk who belongs to a mobster family. The mobster's actions lead to a series of events that set John Wick on a path of revenge and conflict with the Russian mafia.",
 'contexts': [": 20\nReview: After resolving his issues with the Russian mafia, John Wick (Keanu Reeves) returns home. But soon the mobster Santino D'Antonio (Riccardo Scamarcio) visits him to show Wick's marker and tells that he needs to help. John Wicks refuses since he is retired and Santino blows-up his house. John Wick meets the owner of the Continental hotel in New York City, Winston (Ian McShane), and he tells that Wick cannot violate the Mafia rules and shall honor the marker. Santino asks John Wick to kill his sister Gianna D'Antonio (Claudia Gerini) in Rome so that he ca

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [ ]:
def run_evaluation(retrieval_chain):
    answers = []
    contexts = []

    for question in test_questions:
        response = retrieval_chain.invoke({"question" : question})
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])
    response_dataset = Dataset.from_dict({
        "question" : test_questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : test_groundtruths
    })
    return evaluate(response_dataset, metrics)

In [50]:
naive_results = run_evaluation(naive_retrieval_chain)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [38]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"Advanced Rag - {uuid4().hex[0:8]}"

In [ ]:
# !pip install rank_bm25

In [43]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

In [44]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [45]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, opinions on John Wick seem to vary. Some people really enjoyed the movie, while others found it lacking. So, it's safe to say that not everyone generally liked John Wick."

In [48]:
bm25_results = run_evaluation(bm25_retrieval_chain)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [49]:
bm25_results

{'faithfulness': 0.7566, 'answer_relevancy': 0.8711, 'context_recall': 0.7833, 'context_precision': 0.7472, 'answer_correctness': 0.6322}

### Contextual Compression Retrieval
This would NOT run in this notebook even after setting API key.
This runs in the original notebook 

In [53]:
import getpass
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [55]:
# from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
# from langchain_cohere import CohereRerank

# compressor = CohereRerank(model="rerank-english-v3.0")
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=naive_retriever
# )

# contextual_compression_retrieval_chain = (
#     {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
#     | RunnablePassthrough.assign(context=itemgetter("context"))
#     | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
# )

In [56]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [57]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [58]:
multi_query_results = run_evaluation(multi_query_retrieval_chain)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [59]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [60]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", 
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), 
    client=client
)

/var/folders/zx/1kbq8dj123q1cq4b0tqqc64r0000gn/T/ipykernel_13639/1683252106.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  parent_document_vectorstore = Qdrant(


In [61]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [62]:
parent_document_retriever.add_documents(parent_docs, ids=None)

In [63]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [64]:
parent_results = run_evaluation(parent_document_retrieval_chain)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [66]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

In [67]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [68]:
ensemble_results = run_evaluation(ensemble_retrieval_chain)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [69]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

In [70]:
semantic_documents = semantic_chunker.split_documents(documents)

In [71]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

In [72]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

In [73]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [74]:
semantic_results = run_evaluation(semantic_retrieval_chain)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [75]:
print(naive_results)
print(bm25_results)
print("Contextual Compression Results Not Available")
print(multi_query_results)
print(parent_results)
print(ensemble_results)
print(semantic_results)

{'faithfulness': 0.8351, 'answer_relevancy': 0.8763, 'context_recall': 0.9750, 'context_precision': 0.7180, 'answer_correctness': 0.6586}
{'faithfulness': 0.7566, 'answer_relevancy': 0.8711, 'context_recall': 0.7833, 'context_precision': 0.7472, 'answer_correctness': 0.6322}
Contextual Compression Results Not Available
{'faithfulness': 0.8052, 'answer_relevancy': 0.7770, 'context_recall': 0.9750, 'context_precision': 0.6986, 'answer_correctness': 0.6413}
{'faithfulness': 0.8115, 'answer_relevancy': 0.9062, 'context_recall': 0.8500, 'context_precision': 0.7875, 'answer_correctness': 0.6902}
{'faithfulness': 0.8350, 'answer_relevancy': 0.8707, 'context_recall': 0.9750, 'context_precision': 0.7538, 'answer_correctness': 0.6526}
{'faithfulness': 0.8983, 'answer_relevancy': 0.8848, 'context_recall': 0.8750, 'context_precision': 0.7269, 'answer_correctness': 0.6125}


In [78]:
import pandas as pd
df_naive = pd.DataFrame(list(naive_results.items()), columns=['Metric', 'Naive'])
df_bm25 = pd.DataFrame(list(bm25_results.items()), columns=['Metric', 'BM25'])
df_multiQ = pd.DataFrame(list(multi_query_results.items()), columns=['Metric', 'MultiQ'])
df_parent = pd.DataFrame(list(parent_results.items()), columns=['Metric', 'Parent'])
df_ensemble = pd.DataFrame(list(ensemble_results.items()), columns=['Metric', 'Ensemble'])
df_semantic = pd.DataFrame(list(semantic_results.items()), columns=['Metric', 'Semantic'])
df_merged = df_naive.merge(df_bm25, on='Metric').merge(df_multiQ, on='Metric').merge(df_parent, on='Metric')
df_merged = df_merged.merge(df_ensemble, on='Metric').merge(df_semantic, on='Metric')
df_merged

,Metric,Naive,BM25,MultiQ,Parent,Ensemble,Semantic
0,faithfulness,0.835119,0.756566,0.805218,0.811548,0.835000,0.898333
1,answer_relevancy,0.876268,0.871070,0.777026,0.906232,0.870695,0.884772
2,context_recall,0.975000,0.783333,0.975000,0.850000,0.975000,0.875000
3,context_precision,0.717995,0.747222,0.698561,0.787500,0.753800,0.726944
4,answer_correctness,0.658612,0.632169,0.641251,0.690186,0.652633,0.612460
